## Get Weather Prediction

In [13]:
import requests as rq
from bs4 import BeautifulSoup as BS
import numpy as np
import os

In [182]:
# Houston
url = "https://forecast.weather.gov/MapClick.php?w0=t&w3=sfcwind&w4=sky&w5=pop&w6=rh&w7=rain&w8=thunder&w9=snow&w10=fzg&w11=sleet&AheadHour=0&Submit=Submit&&FcstType=digital&textField1=29.7606&textField2=-95.3697&site=all"
# Dallas
# url = 
# San Antonio
# url = 
page = rq.get(url)

In [183]:
html = BS(page.content, 'html.parser')

In [188]:
date_tr = html.find_all('hr')[1].parent.parent.findNext('tr').findNext('tr')
date_tr

<tr align="center"><td align="left" class="date"><font size="1"><b>Hour (CDT)</b></font></td><td class="date"><font size="1"><b>23</b></font></td><td class="date"><font size="1"><b>00</b></font></td><td class="date"><font size="1"><b>01</b></font></td><td class="date"><font size="1"><b>02</b></font></td><td class="date"><font size="1"><b>03</b></font></td><td class="date"><font size="1"><b>04</b></font></td><td class="date"><font size="1"><b>05</b></font></td><td class="date"><font size="1"><b>06</b></font></td><td class="date"><font size="1"><b>07</b></font></td><td class="date"><font size="1"><b>08</b></font></td><td class="date"><font size="1"><b>09</b></font></td><td class="date"><font size="1"><b>10</b></font></td><td class="date"><font size="1"><b>11</b></font></td><td class="date"><font size="1"><b>12</b></font></td><td class="date"><font size="1"><b>13</b></font></td><td class="date"><font size="1"><b>14</b></font></td><td class="date"><font size="1"><b>15</b></font></td><td cl

In [196]:
# Get hours.
hours = []
for item in html.find_all('table')[7].find_all('tr')[2].find_all('td', class_='date'):
    hours.append(item.find('b').get_text())
    
# The second row.
date_tr = html.find_all('hr')[2].parent.parent.findNext('tr').findNext('tr')
tds = date_tr.find_all('td')
for ind in np.arange(1, len(tds)):
    hours.append(tds[ind].get_text())
    
hours = hours[1:]

In [197]:
hours

['23',
 '00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22']

In [198]:
# Get temperatures
temps = []
for item in html.find_all('table')[7].find_all('tr')[3].find_all('td'):
    temps.append(item.get_text())
    
# The second row.
temps_tr = date_tr.findNext('tr')
tds = temps_tr.find_all('td')
for ind in np.arange(1, len(tds)):
    temps.append(tds[ind].get_text())
    
temps = temps[1:]

In [199]:
assert(len(hours) == len(temps))

In [200]:
temps[25:]

['62',
 '60',
 '60',
 '59',
 '59',
 '59',
 '59',
 '62',
 '63',
 '65',
 '68',
 '71',
 '74',
 '76',
 '78',
 '79',
 '79',
 '78',
 '76',
 '75',
 '73',
 '73',
 '72']

In [143]:
# Get temperatures for the next day.
ind = 0
while hours[ind] != '01':
    ind += 1
start = ind
ind += 1

while hours[ind] != '01':
    ind += 1
end = ind

In [144]:
def print_list(l):
    print('[',end="")
    print(l[0],end="")
    for ind in np.arange(1,len(l)):
        print(','+str(l[ind]),end='')
    print(']')

In [145]:
hours = hours[start:end]
hours = [int(x) for x in hours]
temps = temps[start:end]
temps = [int(x) for x in temps]
print_list(hours)
print_list(temps)

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0]
[63,61,60,59,58,59,58,58,61,63,66,69,70,72,75,75,74,72,66,65,65,65,65,64]


## Get Historic Temperature

In [167]:
# Houston
url = "https://w1.weather.gov/data/obhistory/KHOU.html"
# Dallas
# url = 
# San Antonio
# url = 
page = rq.get(url)
html = BS(page.content, 'html.parser')

In [175]:
trs = html.find_all('table')[3].find_all('tr')

In [176]:
date = []
times = []
temps = []
for item in trs[4:-3]:
    tds = item.find_all('td')
    date.append(int(tds[0].get_text()))
    time = tds[1].get_text().split(':')
    # Make sure the minutes are larger than 30
    assert(int(time[1]) > 30)
    # Make sure current time is one hour forward
    hour = int(time[0])
    hour += 1
    if hour == 24:
        hour = 0
    if len(times) > 0:
        assert((hour+1 == times[-1]) or (hour == 23 and times[-1] == 0))
    times.append(hour)
    temps.append(tds[6].get_text())

In [180]:
assert(len(date) == len(times) and len(times) == len(temps))

## Crawl temperatures from weather.com

Not easy to crawl

In [21]:
with open('electricity_data/NW/historic data/seattle_ibm.txt', 'r') as file:
    forecast = file.read()
    forecast = forecast.split('\n')
    # locate the position of the 1:00 AM
    i = 0
    while forecast[i] != '1:00 AM':
        i += 5
    temps = []
    temps.append(forecast[i+2].split('\t')[1][:-1])
    i += 5
    while forecast[i] != '1:00 AM':
        temps.append(forecast[i+2].split('\t')[1][:-1])
        i += 5
    assert(len(temps) == 24)
    print(temps)

['39', '38', '37', '37', '36', '36', '36', '36', '38', '41', '44', '46', '48', '50', '51', '51', '50', '48', '47', '46', '45', '44', '42']


In [16]:
forecast

['5:15 PM',
 'SUN',
 'Sunny\t53°\t51°',
 '0%',
 '42%\tN 6 mph',
 '6:00 PM',
 'SUN',
 'Clear\t49°\t47°',
 '0%',
 '45%\tN 6 mph',
 '7:00 PM',
 'SUN',
 'Clear\t48°\t46°',
 '0%',
 '50%\tNNE 5 mph',
 '8:00 PM',
 'SUN',
 'Clear\t46°\t44°',
 '0%',
 '57%\tNE 3 mph',
 '9:00 PM',
 'SUN',
 'Clear\t43°\t43°',
 '0%',
 '69%\tENE 1 mph',
 '10:00 PM',
 'SUN',
 'Clear\t42°\t42°',
 '0%',
 '74%\tE 1 mph',
 '11:00 PM',
 'SUN',
 'Clear\t40°\t40°',
 '5%',
 '77%\tE 1 mph',
 '12:00 AM',
 'MON',
 'Clear\t39°\t39°',
 '5%',
 '80%\tE 1 mph',
 '1:00 AM',
 'MON',
 'Mostly Clear\t39°\t39°',
 '5%',
 '81%\tE 1 mph',
 '2:00 AM',
 'MON',
 'Mostly Clear\t38°\t38°',
 '5%',
 '81%\tESE 2 mph',
 '3:00 AM',
 'MON',
 'Partly Cloudy\t37°\t37°',
 '5%',
 '84%\tESE 1 mph',
 '4:00 AM',
 'MON',
 'Partly Cloudy\t37°\t37°',
 '5%',
 '84%\tESE 1 mph',
 '5:00 AM',
 'MON',
 'Mostly Cloudy\t36°\t36°',
 '5%',
 '84%\tESE 2 mph',
 '6:00 AM',
 'MON',
 'Mostly Cloudy\t36°\t36°',
 '5%',
 '85%\tESE 1 mph',
 '7:00 AM',
 'MON',
 'Mostly Cloudy\t36°